In [ ]:
# FAZ CONEXÃO COM O S3 DA AWS NA NUVEM 

access_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
secret_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

In [ ]:
# CONFIGURA, PARA O HADOOP, A REGIÃO DE ACESSO PARA O S3 DA AWS NA NUVEM

aws_region = "us-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

In [ ]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD = sc.textFile("s3a://arq-ipca-processeds3/")
myRDD2 = sc.textFile("s3a://arquivos-pmc-processeds3-uf/")
myRDD3 = sc.textFile("s3a://arquivos-pmc-processeds3-catcom/")

myRDD.count()
dbutils.fs.ls("s3a://arq-ipca-processeds3/")

In [ ]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD2.count()
dbutils.fs.ls("s3a://arquivos-pmc-processeds3-uf/")

In [ ]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD3.count()
dbutils.fs.ls("s3a://arquivos-pmc-processeds3-catcom/")

In [ ]:
# LENDO O SCHEMA DOS ARQUIVOS PARQUET DE CADA BUCKET

spark.read.format("parquet").load("s3a://arq-ipca-processeds3/")
spark.read.format("parquet").load("s3a://arquivos-pmc-processeds3-uf/")
spark.read.format("parquet").load("s3a://arquivos-pmc-processeds3-catcom/")

In [ ]:
# EXTRAINDO OS DADOS DE IPCA DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK

dfipca = spark.sql("SELECT ano, mes, avg(perc) as perc \
                            FROM parquet.`s3a://arq-ipca-processeds3/` \
                            where mes < 13 \
                            group by ano, mes order by ano, mes")

from pyspark.sql.functions import lit
#dfipca = dfipca.withColumn("id", lit("ipca"))
dfipca.show()

In [ ]:
# TOTALIZANDO O PERCENTUAL DO IPCA POR ANO E GERANDO O GRÁFICO COM DISPLAY

dfipca_ano = dfipca.groupby('ano').sum('perc').sort('ano', ascending=False)

from pyspark.sql.functions import round, col
dfipca_ano = dfipca_ano.select('ano', round(col('sum(perc)'), 2).alias('perc'))

dfipca_ano.select('*').show()
display(dfipca_ano)

In [ ]:
# CONVERTENDO DATAFRAME SPARK PARA DATAFRAM PANDAS GERANDO O GRÁFICO COM DISPLAY

dfPandas = dfipca_ano.toPandas()
display(dfPandas)

In [ ]:
''' 
EXTRAINDO OS DADOS DE PMC(pesquisa mensal de comércio por UF - ibge - percentual de crescimento das vendas no varejo), 
DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK
'''

dfpmcUF = spark.sql("SELECT uf, ano, mes, avg(m3mensal) as perc \
                            FROM parquet.`s3a://arquivos-pmc-processeds3-uf/` \
                            where mes < 13 \
                            group by uf, ano, mes order by uf, ano, mes")

from pyspark.sql.functions import lit
#dfpmcUF = dfpmcUF.withColumn("id", lit("uf"))
dfpmcUF.show(99999)

In [ ]:
# TOTALIZANDO O PERCENTUAL DE CRESCIMENTO DE VENDAS POR ANO E UF GERANDO O GRÁFICO COM DISPLAY

dfpmcUF_ano = dfpmcUF.groupby('ano', 'uf').sum('perc')
 
from pyspark.sql.functions import round, col
dfpmcUF_ano = dfpmcUF_ano.select('uf', 'ano', round(col('sum(perc)'), 2).alias('perc')).sort('ano', 'perc', ascending=False)
display(dfpmcUF_ano)

In [ ]:
''' 
EXTRAINDO OS DADOS DE PMC(pesquisa mensal de comércio por categoria de comércio - ibge - percentual de crescimento das vendas no varejo), 
DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK
'''

dfpmcCLCOM = spark.sql("SELECT classe_comercio as classe, ano, mes, avg(m3mensal) as perc \
                            FROM parquet.`s3a://arquivos-pmc-processeds3-catcom/` \
                            where mes < 13 and m3mensal > 0 \
                            group by classe_comercio, ano, mes order by classe_comercio, ano, mes")

from pyspark.sql.functions import lit
#dfpmcCLCOM = dfpmcCLCOM.withColumn("id", lit("clcom"))
dfpmcCLCOM.show(99999)

In [ ]:
# TOTALIZANDO O PERCENTUAL DE CRESCIMENTO DE VENDAS POR ANO E CLASSE DE COMÉRCIO GERANDO O GRÁFICO COM DISPLAY

dfpmcCLCOM_ano = dfpmcCLCOM.groupby('ano', 'classe').sum('perc')
 
from pyspark.sql.functions import round, col
dfpmcCLCOM_ano = dfpmcCLCOM_ano.select('classe', 'ano', round(col('sum(perc)'), 2).alias('perc')).sort('ano', 'perc', ascending=False)
display(dfpmcCLCOM_ano)